In [1]:
import os
import sys
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

In [2]:
def imshow_in_jupyter(img):
    img_plt = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img_plt)

# Function

## feature match

In [3]:
def feature_matching(img1, img2, savefig=False):
     '''
    Detect, extract and match features between img1 and img2.
    Using SIFT as the detector/extractor, but this is inconsequential to the user.

    Returns: (pts1, pts2), where ptsN are points on image N.
        The lists are "aligned", i.e. point i in pts1 matches with point i in pts2.

    Usage example:
        img1 = cv2.imread("image1.jpg", 0)
        img2 = cv2.imread("image2.jpg", 0)
        (pts1, pts2) = feature_matching(img1, img2)

        plt.subplot(121)
        plt.imshow(img1)
        plt.scatter(pts1[:,:,0],pts1[:,:,1], 0.5, c='r', marker='x')
        plt.subplot(122)
        plt.imshow(img2)
        plt.scatter(pts1[:,:,0],pts1[:,:,1], 0.5, c='r', marker='x')
    '''
    # Initiate SIFT detector
    sift = cv2.xfeatures2d.SIFT_create()
    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(img1,None)
    kp2, des2 = sift.detectAndCompute(img2,None)
    # FLANN parameters
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)   # or pass empty dictionary
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    matches2to1 = flann.knnMatch(des2,des1,k=2)

    matchesMask_ratio = [[0,0] for i in xrange(len(matches2to1))]
    match_dict = {}
    for i,(m,n) in enumerate(matches2to1):
        if m.distance < 0.7*n.distance:
            matchesMask_ratio[i]=[1,0]
            match_dict[m.trainIdx] = m.queryIdx

    good = []
    recip_matches = flann.knnMatch(des1,des2,k=2)
    matchesMask_ratio_recip = [[0,0] for i in xrange(len(recip_matches))]

    for i,(m,n) in enumerate(recip_matches):
        if m.distance < 0.7*n.distance: # ratio
            if m.queryIdx in match_dict and match_dict[m.queryIdx] == m.trainIdx: #reciprocal
                good.append(m)
                matchesMask_ratio_recip[i]=[1,0]

    if savefig:
        draw_params = dict(matchColor = (0,255,0),
                           singlePointColor = (255,0,0),
                           matchesMask = matchesMask_ratio_recip,
                           flags = 0)
        img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,recip_matches,None,**draw_params)

        plt.figure(),plt.xticks([]),plt.yticks([])
        plt.imshow(img3,)
        plt.savefig("feature_matching.png",bbox_inches='tight')

    return ([ kp1[m.queryIdx].pt for m in good ],[ kp2[m.trainIdx].pt for m in good ])

## cylindrical Warp
Warp an image from cartesian coordinates (x, y) into cylindrical coordinates (theta, h)


In [ ]:
def cylindricalWarpImage(img1, K, savefig=False):
    '''
    Warp an image from cartesian coordinates (x, y) into cylindrical coordinates (theta, h)
    Returns: (image, mask)
    Mask is [0,255], and has 255s wherever the cylindrical images has a valid value.
    Masks are useful for stitching

    Usage example:

        im = cv2.imread("myimage.jpg",0) #grayscale
        h,w = im.shape
        f = 700
        K = np.array([[f, 0, w/2], [0, f, h/2], [0, 0, 1]]) # mock calibration matrix
        imcyl = cylindricalWarpImage(im, K)
    '''
    
    f = K[0,0]

    im_h,im_w = img1.shape

    # go inverse from cylindrical coord to the image
    # (this way there are no gaps)
    cyl = np.zeros_like(img1)
    cyl_mask = np.zeros_like(img1)
    cyl_h,cyl_w = cyl.shape
    x_c = float(cyl_w) / 2.0
    y_c = float(cyl_h) / 2.0
    for x_cyl in np.arange(0,cyl_w):
        for y_cyl in np.arange(0,cyl_h):
            theta = (x_cyl - x_c) / f
            h     = (y_cyl - y_c) / f
            X = np.array([math.sin(theta), h, math.cos(theta)])
            X = np.dot(K,X)
            x_im = X[0] / X[2]
            if x_im < 0 or x_im >= im_w:
                continue

            y_im = X[1] / X[2]
            if y_im < 0 or y_im >= im_h:
                continue

            cyl[int(y_cyl),int(x_cyl)] = img1[int(y_im),int(x_im)]
            cyl_mask[int(y_cyl),int(x_cyl)] = 255


    if savefig:
        plt.imshow(cyl, cmap='gray')
        plt.savefig("cyl.png",bbox_inches='tight')

    return (cyl,cyl_mask)

In [ ]:
def getTransform(src, dst, method='affine'):
    
    '''
    Calculate the geometric transform (only affine or homography) between two images,
    based on feature matching and alignment with a robust estimator (RANSAC).

    Returns: (M, pts1, pts2, mask)
    Where: M    is the 3x3 transform matrix
           pts1 are the matched feature points in image 1
           pts2 are the matched feature points in image 2
           mask is a binary mask over the lists of points that selects the transformation inliers

    Usage example:
        img1 = cv2.imread("image1.jpg", 0)
        img2 = cv2.imread("image2.jpg", 0)
        (M, pts1, pts2, mask) = getTransform(img1, img2)

        # for example: transform img1 to img2's plane
        # first, make some room around img2
        img2 = cv2.copyMakeBorder(img2,200,200,500,500, cv2.BORDER_CONSTANT)
        # then transform img1 with the 3x3 transformation matrix
        out = cv2.warpPerspective(img1, M, (img2.shape[1],img2.shape[0]), dst=img2.copy(), borderMode=cv2.BORDER_TRANSPARENT)

        plt.imshow(out, cmap='gray')
        plt.show()
    '''
    pts1,pts2 = feature_matching(src,dst)

    src_pts = np.float32(pts1).reshape(-1,1,2)
    dst_pts = np.float32(pts2).reshape(-1,1,2)

    if method == 'affine':
        M, mask = cv2.estimateAffine2D(src_pts, dst_pts, cv2.RANSAC, ransacReprojThreshold=5.0)
        M = np.append(M, [[0,0,1]], axis=0)

    if method == 'homography':
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

    matchesMask = mask.ravel().tolist()

    return (M, pts1, pts2, mask)

## Perspective Warping

In [ ]:
def Perspective_warping(img1, img2, img3):
    '''
    perspective warping
    Warp img2 and img3 on img1
    '''
    # first, make some room around img1
    img1 = cv2.copyMakeBorder(img1,200,200,500,500, cv2.BORDER_CONSTANT)

    # then transform img2,3 with the 3x3 transformation matrix to img1 plane
    (M21, pts2, pts1, mask) = getTransform(img2, img1,method='homography')
    out = cv2.warpPerspective(img2, M21, (img1.shape[1],img1.shape[0]), dst=img1.copy(),flags = cv2.INTER_LINEAR,borderMode=cv2.BORDER_TRANSPARENT)

    (M31, pts3, pts1, mask) = getTransform(img3, img1,method='homography')
    output_image = cv2.warpPerspective(img3, M31, (img1.shape[1],img1.shape[0]), dst=out,flags = cv2.INTER_LINEAR,borderMode=cv2.BORDER_TRANSPARENT)

    output_name = sys.argv[5] + "output_homography.png"
    cv2.imwrite(output_name, output_image)

    return output_image

## Cynlindrical Warping

In [ ]:
def Cylindrical_warping(img1, img2, img3):
    '''
    warp img1,2,3 to a Cynlinder
    '''
    
    h1,w1 = img1.shape
    f = 420
    K1 = np.array([[f, 0, w1/2], [0, f, h1/2], [0, 0, 1]]) # mock calibration matrix
    imcyl_1, mask1 = cylindricalWarpImage(img1, K1)

    h2,w2 = img2.shape
    f = 420
    K2 = np.array([[f, 0, w2/2], [0, f, h2/2], [0, 0, 1]]) # mock calibration matrix
    imcyl_2, mask2 = cylindricalWarpImage(img2, K2)

    h3,w3 = img3.shape
    f = 420
    K3 = np.array([[f, 0, w3/2], [0, f, h3/2], [0, 0, 1]]) # mock calibration matrix
    imcyl_3, mask3 = cylindricalWarpImage(img3, K3)

    imcyl_1 = cv2.copyMakeBorder(imcyl_1,50,50,300,300, cv2.BORDER_CONSTANT)

    (M21, pts2, pts1, mask) = getTransform(imcyl_2,imcyl_1,method='affine')
    M21 = M21[:2,:]
    bg = imcyl_1.copy()
    bg = cv2.bitwise_and(bg,0)
    out2 = cv2.warpAffine(imcyl_2, M21, (imcyl_1.shape[1],imcyl_1.shape[0]),dst=bg.copy(),flags = cv2.INTER_LINEAR,borderMode=cv2.BORDER_CONSTANT)
    out_mask2 = cv2.warpAffine(mask2, M21, (imcyl_1.shape[1],imcyl_1.shape[0]),dst=bg.copy(),flags = cv2.INTER_LINEAR,borderMode=cv2.BORDER_TRANSPARENT)
    mask_inv2 = cv2.bitwise_not(out_mask2)
    plane = imcyl_1.copy()
    img1_bg = cv2.bitwise_and(plane,plane,mask = mask_inv2)
    out12 = cv2.add(img1_bg,out2)

    (M31, pts3, pts1, mask) = getTransform(imcyl_3,imcyl_1,method='affine')
    M31 = M31[:2,:]
    out3 = cv2.warpAffine(imcyl_3, M31, (imcyl_1.shape[1],imcyl_1.shape[0]),dst=bg.copy(),flags = cv2.INTER_LINEAR,borderMode=cv2.BORDER_TRANSPARENT)
    out_mask3 = cv2.warpAffine(mask3, M31, (imcyl_1.shape[1],imcyl_1.shape[0]),dst=bg.copy(),flags = cv2.INTER_LINEAR,borderMode=cv2.BORDER_TRANSPARENT)
    mask_inv3 = cv2.bitwise_not(out_mask3)
    plane12 = out12.copy()
    img12_bg = cv2.bitwise_and(plane12,plane12,mask = mask_inv3)
    output_image = cv2.add(img12_bg,out3)

    output_name = sys.argv[5] + "output_cylindrical.png"
    cv2.imwrite(output_name, output_image)

    return output_img

## Laplacian warping
Waiting...